# Diffusion gradients in Bruker data sets
## or: How I learned to stop worying and love coordinate systems

**WARNING** This is a work in progress. The informations should be completed with
* Data sets with oblique slices from Paravision 6
* Check that the RPF to RPH conversion is valid in other data sets
* Data sets from Paravision 5

The official documentation from Bruker (Operating Manual of Paravision 6.0.1 and *ParaVision Parameters*) are lacking details to understand which coordinate system (gradient, subject, magnet or image) is used when manipulating vectors and matrices. This is especially difficult in DWI acquisitions, where the direction of the diffusion gradient is of the utmost importance in DEC (directionally-encoded color) images and tractography. The documentation is even self-contradicting for some fields: `ACQ_grad_matrix` is described as the transformation from gradient coordinates to subject coordinates in the Operating Manual (section 3.2.6.10.1) and from the gradient coordinates to patient-indenpendent coordinates in *ParaVision Parameters* (page D-2-36).

Based on a diffusion data set provided by @MathieuSantin (readout gradient from head to tail, phase on the left-right axis with undefined direction, and slice on the ventro-dorsal axis also with undefined direction), we try to understand how to convert between different coordinate systems and how to express the directions of the diffusion gradient in these systems. In addition to core Python modules, [Dicomifier](https://github.com/lamyj/dicomifier/) and [NumPy](http://www.numpy.org/) are required.

The following anatomical orientations are used in this document:
* *L*, *R*: left and right of the subject
* *D*, *V*: dorsal (anterior or back) and ventral (posterior or front) sides of the subject
* *F*, *H*: foot (inferior or caudal) and head (superior or rostral) extremities of the subject

In [1]:
import os
import sys

import dicomifier
import numpy

/Users/julien/src/dicomifier/master/src/python/dicomifier/bruker_to_dicom/image.py:13: RuntimeWarning: to-Python converter for std::__1::vector<int, std::__1::allocator<int> > already registered; second conversion method ignored.
  import odil


We start by loading the main meta-data files, `acqp`, `method` and `visu_pars`, of the diffusion series.

In [2]:
bruker_series = "bruker/7"
method = dicomifier.bruker.Dataset()
method.load(os.path.join(bruker_series, "method"))

acqp = dicomifier.bruker.Dataset()
acqp.load(os.path.join(bruker_series, "acqp"))

visu_pars = dicomifier.bruker.Dataset()
visu_pars.load(os.path.join(bruker_series, "pdata", "1", "visu_pars"))

We then define a helper function to fetch n-dimensional numeric arrays from Bruker data sets.

In [3]:
def get_field_data(field, shape=None):
    return numpy.reshape(
        [field.get_float(x) for x in range(numpy.cumprod(field.shape)[-1])], 
        field.shape)

## Coordinate systems

According to the Operating Manual (section 3.2.6.10.1), the gradient coordinate system has a (*readout*, *phase*, *slice*) basis, and the transformation matrix from this coordinate system to the subject coordinate system (i.e. the coordinates of the gradient basis in the subject basis) is stored in `ACQ_grad_matrix`. The equation in the aforementioned section uses a $v\cdot M$ formalism, which points to the use of row-vectors, and ParaVision Parameters (pages D-2-36 and D-2-65) mentions matrices stored in the row-major order. The subject coordinate system is described by Bruker (Operating Manual, sections 1.3.6.1, 1.3.6.2, and 3.2.6.10.1) to be (*L* &rarr; *R*, *V* &rarr; *D*, *F* -> *H*); note that this is a [left-handed](https://en.wikipedia.org/wiki/Right-hand_rule) coordinate system.

In [4]:
# WARNING: ACQ_grad_matrix is an array of matrices
ACQ_grad_matrix = get_field_data(acqp.get_field("ACQ_grad_matrix"))[0]
print(
    "In subject coordinates,\n"
    "   read: {}\n"
    "  phase: {}\n"
    "  slice: {}".format(
        ACQ_grad_matrix[0], ACQ_grad_matrix[1], ACQ_grad_matrix[2]))

In subject coordinates,
   read: [ 0.  0.  1.]
  phase: [ 1.  0.  0.]
  slice: [ 0.  1.  0.]


With the exception of an inverted readout gradient (which should be $(0, 0, -1)$ to match the head-to-tail direction) and a possible inversion of the other axes, it matches the original description. This *F*-*H* inversion will appear in further expressions, and indicates that `ACQ_grad_matrix` converts to the *RDF* subject coordinate system, and not the the *RDH* as written by Bruker. In order to increase readability, we define this transformation matrix (which is its own inverse).

In [5]:
rdf_to_rdh = numpy.asarray([
    [ 1, 0,  0], 
    [ 0, 1,  0], 
    [ 0, 0, -1]])

The position matrices, converting from the subject RDF coordinate system to the magnet (or world) coordinate system, are described in ParaVision Parameters (page D-2-33).

In [6]:
position_matrix = {
    "Feet_Supine": numpy.asarray([[ 1, 0, 0], [0,  1, 0], [0, 0,  1]]),
    "Head_Supine": numpy.asarray([[-1, 0, 0], [0,  1, 0], [0, 0, -1]]),
    "Feet_Prone":  numpy.asarray([[-1, 0, 0], [0, -1, 0], [0, 0,  1]]),
    # ...
}
ACQ_patient_pos = position_matrix[acqp.get_field("ACQ_patient_pos").get_string(0)]

Since `Feet_Supine` corresponds to an identity position matrix in the RDF subject coordinates, magnet axes are (when facing the tunnel):
* Left &rarr; Right
* Up &rarr; Down
* Front &rarr; Back

The last coordinate system is the image one where the axes are those of the voxel block. The transformation matrix from the subject coordinate system to the image coordinate system is stored in `VisuCoreOrientation`. It transforms from the *LDH* system in Paravision 6 and from the *RVH* system in Paravision 5 (Paravision Parameters, pages D-2-64 and D-2-65).

In [7]:
VisuCoreOrientation = get_field_data(visu_pars.get_field("VisuCoreOrientation"))
VisuCoreOrientation = VisuCoreOrientation.reshape(3,3)
print(
    "In image coordinates,\n"
    "   R->L: {}\n"
    "   V->D: {}\n"
    "   F->H: {}".format(
        VisuCoreOrientation[0], VisuCoreOrientation[1], VisuCoreOrientation[2]))

In image coordinates,
   R->L: [ 1.  0.  0.]
   V->D: [ 0.  0. -1.]
   F->H: [ 0.  1.  0.]


These axes can be checked by displaying the converted NIfTI image in FSLeyes or ITK-SNAP. Note that since the subject was positioned as head-first prone but but registered in Paravision as head-first supine, some axes might be flipped.

## Diffusion gradient

The directions of the diffusion gradient are stored in the gradient coordinate system in `PVM_DwDir`, unless `PVM_DwDirectScale` is set to `Yes`; in that case, they are stored in the "x,y,z-coordinate system" (Operating Manual, section 1.9.1.2). This latter system is used to refer to the magnet coordinate system. They are also stored in the subject coordinate system in `VisuAcqDiffusionBMatrix` (ParaVision Parameters, page D-2-72). However, this field does not directly store the diffusion gradient directions, but the [b-matrix](http://doi.org/10.1006/jmrb.1994.1037). In this formalism, the diffusion gradient is colinear to the eigenvector associated with the eigenvalue having the largest absolute value of the b-matrix (**warning** need to find a reference for this). Moreover, `VisuAcqDiffusionBMatrix` also include the volumes acquired without the diffusion gradient; in the Saimiri data set, these volumes are at the beginning of the acquisitions, and their count is stored in `PVM_DwAoImages`.


In [8]:
PVM_DwDir = get_field_data(method.get_field("PVM_DwDir"))

PVM_DwAoImages = method.get_field("PVM_DwAoImages").get_int(0)

VisuAcqDiffusionBMatrix = get_field_data(
    visu_pars.get_field("VisuAcqDiffusionBMatrix"))
VisuAcqDiffusionBMatrix = numpy.reshape(VisuAcqDiffusionBMatrix, (-1, 3, 3))

To check the colinearity of `PVM_DwDir` transformed to the subject coordinate system and the first eigenvector of `VisuAcqDiffusionBMatrix`, we use the absolute value of the scalar product. The diffusion directions in the gradient coordinate system are transformed in the RDF coordinate system using `ACQ_grad_matrix`, and then to the RDH coordinate system used by `VisuAcqDiffusionBMatrix`.

In [9]:
def get_orientation_and_b_value(b_matrix):
    eigenvalues, eigenvectors = numpy.linalg.eigh(b_matrix)
    eigenvalues = numpy.abs(eigenvalues)
    return eigenvectors[:,numpy.argmax(eigenvalues)], eigenvalues.max()

directions_subject = [
    get_orientation_and_b_value(x)[0]
    for x in VisuAcqDiffusionBMatrix[PVM_DwAoImages:]]

transform = numpy.dot(ACQ_grad_matrix, rdf_to_rdh)

difference = [
    numpy.abs(numpy.dot(numpy.dot(x, transform), y))
    for x, y in zip(PVM_DwDir, directions_subject)]
difference = numpy.round(difference, 6)
print(
    "Absolute value of dot-product "
    "between transformed PVM_DwDir and VisuAcqDiffusionBMatrix: {} - {}".format(
        min(difference), max(difference)))

Absolute value of dot-product between transformed PVM_DwDir and VisuAcqDiffusionBMatrix: 0.998381 - 1.0


The content of `VisuAcqDiffusionBMatrix` is also stored in the undocumented `PVM_DwBMatPat`. 

In [10]:
PVM_DwBMatPat = get_field_data(method.get_field("PVM_DwBMatPat"))
print(
    "Maximum difference between "
    "VisuAcqDiffusionBMatrix and PVM_DwBMatPat: {}".format(
        numpy.abs(PVM_DwBMatPat-VisuAcqDiffusionBMatrix).max()))

Maximum difference between VisuAcqDiffusionBMatrix and PVM_DwBMatPat: 0.0


Two other undocumented fields store the b-matrices in the magnet and image coordinate systems, `PVM_DwBMatMag` and `PVM_DwBMatImag`.

In [11]:
PVM_DwBMatMag = get_field_data(method.get_field("PVM_DwBMatMag"))
directions_magnet = [
    get_orientation_and_b_value(x)[0]
    for x in PVM_DwBMatMag[PVM_DwAoImages:]]
# The transform is also stored in PVM_SliceGeo
transform = numpy.dot(ACQ_grad_matrix, ACQ_patient_pos)
difference = [
    numpy.abs(numpy.dot(numpy.dot(x, transform), y))
    for x, y in zip(PVM_DwDir, directions_magnet)]
difference = numpy.round(difference, 6)
print(
    "Absolute value of dot-product "
    "between transformed PVM_DwDir and PVM_DwBMatMag: {} - {}".format(
        min(difference), max(difference)))

Absolute value of dot-product between transformed PVM_DwDir and PVM_DwBMatMag: 0.998381 - 1.0


In [12]:
PVM_DwBMatImag = get_field_data(method.get_field("PVM_DwBMatImag"))
directions_image = [
    get_orientation_and_b_value(x)[0]
    for x in PVM_DwBMatImag[PVM_DwAoImages:]]

rdf_to_ldh = numpy.asarray([
    [-1, 0,  0], 
    [ 0, 1,  0], 
    [ 0, 0, -1]])
transform = numpy.dot(
    numpy.dot(ACQ_grad_matrix, rdf_to_ldh),
    VisuCoreOrientation)

difference = [
    numpy.abs(numpy.dot(numpy.dot(x, transform), y))
    for x, y in zip(PVM_DwDir, directions_image)]
difference = numpy.round(difference, 3)
print(
    "Absolute value of dot-product "
    "between transformed PVM_DwDir and PVM_DwBMatImag: {} - {}".format(
        min(difference), max(difference)))

Absolute value of dot-product between transformed PVM_DwDir and PVM_DwBMatImag: 0.998 - 1.0


## Interoperability with other standards and diffusion softwares

In [DICOM](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.13.5.9.html), the direction of the diffusion gradient and the b-matrix are explicitely specified in the [LDH](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.7.6.2.html#sect_C.7.6.2.1.1) subject coordinate system. 

We can now create the DICOM meta-data containing the diffusion information.

In [13]:
rdh_to_ldh = numpy.asarray([
    [-1, 0, 0],
    [ 0, 1, 0],
    [ 0, 0, 1]])
meta_data = {
    "MRDiffusionSequence": []}
for matrix in VisuAcqDiffusionBMatrix:
    orientation, b_value = get_orientation_and_b_value(matrix)
    meta_data["MRDiffusionSequence"].append({
        "DiffusionBValue": [b_value],
        "DiffusionGradientDirectionSequence": [{ 
            "DiffusionGradientOrientation": numpy.dot(orientation, rdh_to_ldh) }],
    })

### FSL: `bvecs` and `bvals` files

FSL uses a [radiological voxel convention](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FDT/FAQ#What_conventions_do_the_bvecs_use.3F). This rather convoluted expression means that the directions are in the image coordinate system if the image-to-subject transformation has a negative determinant. If this determinant is positive, then an additional flip on the x axis must be applied to match the "old Analyze convention". As an extra indication that the directions are stored in the image coordinate system and not in the subject coordinate system, both the [Eddy FAQ](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/Faq#Will_eddy_rotate_my_bvecs_for_me.3F) and a [practical in the FSL course](http://fsl.fmrib.ox.ac.uk/fslcourse/lectures/practicals/fdt1/index.html) mention rotation of the direction when performing motion correction, i.e. being image dependent. MRtrix also [applies this process](https://github.com/MRtrix3/mrtrix3/blob/master/src/dwi/gradient.cpp#L127-L133) the other way around to convert from the `bvecs` file to their gradient scheme.

We first compute the NIfTI orientation matrix from `VisuCoreOrientation`, using the more common column-vector formalism.

In [14]:
nifti_orientation_matrix = numpy.linalg.inv(VisuCoreOrientation).T
ldh_to_rvh = numpy.asarray([
    [-1,  0, 0],
    [ 0, -1, 0],
    [ 0,  0, 1]])
nifti_orientation_matrix = numpy.dot(ldh_to_rvh, nifti_orientation_matrix)

The `bvecs` and `bvals` are then given by:

In [15]:
bvecs = []
bvals = []
for entry in meta_data["MRDiffusionSequence"]:
    # In LDH
    orientation = entry["DiffusionGradientDirectionSequence"][0]["DiffusionGradientOrientation"]
    # To RVH
    orientation = numpy.dot(ldh_to_rvh, orientation)
    # To image
    orientation = numpy.dot(nifti_orientation_matrix, orientation)
    # To Analyze convention, if required
    if numpy.linalg.det(nifti_orientation_matrix) > 0:
        orientation[0] *= -1
    
    bvecs.append(orientation)
    bvals.append(entry["DiffusionBValue"][0])

from io import BytesIO
with BytesIO() as fd:
    numpy.savetxt(fd, numpy.transpose(bvecs))
    print(fd.getvalue())
with BytesIO() as fd:
    numpy.savetxt(fd, numpy.reshape(bvals, (1, -1)))
    print(fd.getvalue())

-5.768675629356625478e-01 -5.768675629356625478e-01 -5.768675629356625478e-01 -5.768675629356625478e-01 -5.768675629356625478e-01 -5.768675629356625478e-01 -9.393194955829210768e-02 -7.783056238775881708e-02 -2.631065794850782624e-01 2.051724698866855823e-01 3.253394028687053297e-01 -3.484198060584335344e-01 -6.279201051995858540e-02 2.716175364251311453e-01 5.025201770021500147e-01 3.218936613367740507e-01 -7.636506318667542204e-02 -5.884590213621501142e-01 5.996751087900475952e-01 3.200139459086751570e-01 2.867061299336806646e-01 5.473831400252617829e-01 -7.318592792727741658e-01 6.028294788629695589e-01 -9.833268368430971018e-02 2.610071351222708058e-01 5.532088258674759951e-01 -8.096787077704203917e-01 5.615007321558193043e-01 2.416952450381537054e-01 -8.881409488149426268e-02 5.641019855844126019e-01 7.876663087936293106e-01 8.260385869616144738e-01 3.889764631902154490e-01 1.280285346082182529e-01 4.707572281645845402e-01 7.362738181739388876e-01 7.885873579785047660e-01 8.078382

In order to check the correctness of the `bvecs` and `bvals`, we use FSL's own `dtifit`:

```shell
bet dwi.nii.gz brain -m
dtifit -k dwi.nii.gz -r bvec -b bval -o fsl -m bet_mask.nii.gz
fsleyes --layout grid fsl_FA.nii.gz fsl_V1.nii.gz -ot linevector
```

### MRtrix

The [MRtrix gradient file](http://mrtrix.readthedocs.io/en/latest/concepts/dw_scheme.html#mrtrix-format) has conflicting informations about the coordinate system it uses: it mentions both "scanner coordinates" and "as DICOM". However, when [loading NIfTI files](https://github.com/MRtrix3/mrtrix3/blob/master/core/file/nifti2_utils.cpp#L158-L203), only the `sform` or `qform` is used to establish the [MRtrix image transform](https://github.com/MRtrix3/mrtrix3/blob/master/core/file/nifti2_utils.cpp#L158-L203). Diffusion data from NIfTI is [loaded](https://github.com/MRtrix3/mrtrix3/blob/master/src/dwi/gradient.cpp#L98-L152) by transform the `bvec` data to the RAS subject coordinate system.

In [16]:
orientation_mrtrix = [
    numpy.dot(
        ldh_to_rvh, 
        x["DiffusionGradientDirectionSequence"][0]["DiffusionGradientOrientation"]) 
    for x in meta_data["MRDiffusionSequence"]]
mrtrix_scheme = numpy.hstack((
    orientation_mrtrix, [x["DiffusionBValue"] for x in meta_data["MRDiffusionSequence"]]))
with BytesIO() as fd:
    numpy.savetxt(fd, mrtrix_scheme)
    print(fd.getvalue())

-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-5.768675629356625478e-01 5.783144729862279565e-01 5.768675629356627699e-01 1.769846906816620091e+01
-9.393194955829210768e-02 9.905187342488982472e-01 1.002468251574038655e-01 1.006440833614461326e+03
-7.783056238775881708e-02 9.647710344551821970e-01 -2.513150505534483270e-01 1.004720841735005479e+03
-2.631065794850782624e-01 -9.604732785956614505e-01 9.091759420135833714e-02 1.004069099770795333e+03
2.051724698866855823e-01 9.469664678983631401e-01 2.473029847711020190e-01 1.000244677726

Correctness can be checked with `dwi2tensor`:
```shell
dwi2tensor -grad mrtrix.scheme -mask brain_mask.nii.gz dwi.nii.gz mrtrix_DT.nii.gz
tensor2metric -fa mrtrix_FA.nii.gz mrtrix_DT.nii.gz
tensor2metric -vector mrtrix_V1.nii.gz -num 1 mrtrix_DT.nii.gz
mrview mrtrix_FA.nii.gz -fixel.load mrtrix_V1.nii.gz -mode 2
```

### Camino